<a href="https://colab.research.google.com/github/castlechoi/studyingDL/blob/main/NLP/Word2Vec_CBOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing

## Download corpus

In [60]:
import torch
import torch.optim as optim
import torch.nn as nn

import nltk
nltk.download("book", quiet = True)

True

In [61]:
# nltk.book에 저장된 다양한 corpus
from nltk.book import *
nltk.book.texts()

text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [62]:
# tokenize 모두 완료 되어 있음
ex_book = nltk.book.text1[:500]

## Stop-words 제거

In [63]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

# stop-words에 특수기호 추가
stopwords  = stopwords + ['.',',','\'','!','?','\"','[',']','(',')','*','I',':',';','-','."','--','<','>']

print(f'Stop-words의 개수 : {len(stopwords)}')

Stop-words의 개수 : 198


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
# 불용어 빠진 것을 확인
ex_book_no_stopwords = [[t] for t in ex_book if t not in stopwords]
print(f'불용어 제외한 후 문장의 길이 : {len(ex_book_no_stopwords)}')

불용어 제외한 후 문장의 길이 : 241


## Data preprocessing hyperparameter


In [65]:
min_count = 2
window = 2

## One-hot Encoding

In [66]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

In [67]:
# 토큰 집합 추출 ( 등장횟수가 2이하면 토큰화 안함)
cut_off = 0

tokens = pd.Series(ex_book_no_stopwords).value_counts()
for i in range(len(tokens)):
  if tokens[i] == min_count-1:
    cut_off = i
    break
tokens = tokens[:cut_off].index.tolist()
print(f'Token의 개수 : {len(tokens)}')

Token의 개수 : 24


In [68]:
# token에 없는 데이터 모두 <unk>로 변경
ex_book_process = [t if t in tokens else ['<unk>'] for t in ex_book_no_stopwords]

In [69]:
# One-Hot Encoding
oe = OneHotEncoder()
document_matrix = oe.fit_transform(ex_book_process)
print(f'문서의 단의 개수 : {document_matrix.shape[0]}')
print(f'One-Hot vector의 크기 : {document_matrix.shape[1]}')

문서의 단의 개수 : 241
One-Hot vector의 크기 : 25


## GPU 설정

In [70]:
#GPU 체크
is_cuda = torch.cuda.is_available()
if is_cuda:
  device = torch.device("cuda")
  print("GPU is available")
else:
  device = torch.device("cpu")
  print("CPU is availalbe")

GPU is available


# CBOW Low-level

## Train data preprocessing 

In [71]:
# train_x에 CBOW의 input으로 들어가는 4개의 벡터
train_x = []
train_y = []
for i in range(document_matrix.shape[0] - (window * 2)):
  neighbor = []
  neighbor.append(document_matrix[i].toarray())
  neighbor.append(document_matrix[i+1].toarray())
  neighbor.append(document_matrix[i+3].toarray())
  neighbor.append(document_matrix[i+4].toarray())

  train_x.append(neighbor)
  train_y.append(document_matrix[i+2].toarray())

In [72]:
train_x_tensor = torch.FloatTensor(train_x).view(-1,4,document_matrix.shape[1]).to(device)
train_y_tensor = torch.FloatTensor(train_y).view(-1,document_matrix.shape[1]).to(device)

print(f'train_x의 shape : {train_x_tensor.shape}') # 단어 개수 * 4 * one_hot
print(f'train_y의 shape : {train_y_tensor.shape}') # 단어 개수 * one_hot

train_x의 shape : torch.Size([237, 4, 25])
train_y의 shape : torch.Size([237, 25])


## Define Model

In [73]:
# Hyperparameter
num_epochs = 5000
lr = 0.001
emb_vector_size = 2

In [74]:
# Define weights without bias
W = torch.randn(document_matrix.shape[1],emb_vector_size).to(device).requires_grad_()
W_prime = torch.randn(emb_vector_size,document_matrix.shape[1]).to(device).requires_grad_()

In [75]:
# Define optimizer and loss
CBOW_optimizer = optim.Adam([W], lr = lr)
CBOW_optimizer_p = optim.Adam([W_prime], lr = 0.001)
criterion = nn.CrossEntropyLoss()

## Train the model

In [76]:
for i in range(num_epochs+1):
  # Input : 4 neighbor vector
  y_pred = train_x_tensor @ W
  y_pred = torch.mean(y_pred, dim = 1)
  
  # Input : Embedding vector
  # Output : predict one-hot vector
  y_pred = y_pred @ W_prime
  y_pred = y_pred.softmax(dim = 1)

  # compute loss
  loss = criterion(y_pred , train_y_tensor)
  
  # initiate optimizer
  CBOW_optimizer.zero_grad()
  CBOW_optimizer_p.zero_grad()
  # backpropagation
  loss.backward()
  CBOW_optimizer.step()
  CBOW_optimizer_p.step()

  if i % 500 == 0:
    print(f'epoch {i} : {loss.item()}')


epoch 0 : 3.1873133182525635
epoch 500 : 2.735260009765625
epoch 1000 : 2.549489974975586
epoch 1500 : 2.532850742340088
epoch 2000 : 2.5290212631225586
epoch 2500 : 2.5275511741638184
epoch 3000 : 2.5267441272735596
epoch 3500 : 2.525071382522583
epoch 4000 : 2.5167064666748047
epoch 4500 : 2.512925624847412
epoch 5000 : 2.5112602710723877


## Print lookuptable

In [77]:
print(W)
print(W_prime)

tensor([[-1.5209, -0.5212],
        [-2.2339,  1.9593],
        [-1.0744,  2.1118],
        [-2.8682,  0.6956],
        [-2.9756,  0.4261],
        [-0.9636,  2.1027],
        [-2.3552,  0.7953],
        [-1.8424,  0.4236],
        [-3.3176,  1.3479],
        [-4.3817, -2.1823],
        [ 0.5047,  3.1857],
        [-2.7700,  2.1188],
        [-1.9593,  0.9386],
        [-1.3785,  2.4849],
        [ 4.1758,  0.4925],
        [-1.2312,  2.9900],
        [-1.5026,  2.3819],
        [-0.4630,  1.3901],
        [-0.7521,  2.9094],
        [-3.8063,  0.5985],
        [-1.8714,  4.0951],
        [-2.5592,  0.3975],
        [-1.3717, -0.0237],
        [-0.9343,  0.8598],
        [-1.1811,  0.2602]], device='cuda:0', requires_grad=True)
tensor([[ 1.1965, -3.2145,  0.1024,  0.2095,  1.2201,  2.7619,  2.4853,  2.1779,
          1.0853,  2.3532,  3.5925,  0.1972,  0.1986,  3.0633, -0.8037, -0.1925,
         -0.8907,  0.3639,  1.3655,  2.8989, -0.2639,  1.1151,  0.8050,  1.8778,
          1.1069],


# CBOW Using nn.Linear

## Define Model

In [78]:
# Hyperparameter
num_epochs = 5000
lr = 0.001
emb_vector_size = 2

In [79]:
# Define model, optimizer and loss
CBOW_linear = nn.Linear(document_matrix.shape[1], emb_vector_size, bias = False).to(device)
CBOW_linear_p = nn.Linear(emb_vector_size,document_matrix.shape[1], bias = False).to(device)

CBOW_linear_optimizer = optim.Adam(CBOW_linear.parameters(), lr = lr)
CBOW_linear_optimizer_p = optim.Adam(CBOW_linear_p.parameters(), lr = lr)
criterion = nn.CrossEntropyLoss()

## Train the model

In [80]:
for i in range(num_epochs+1):
  # Input : 4 neighbor vector
  y_pred = CBOW_linear(train_x_tensor)
  y_pred = torch.mean(y_pred, dim = 1)

  # Input : Embedding vector
  # Output : predict one-hot vector
  y_pred = CBOW_linear_p(y_pred)
  y_pred = y_pred.softmax(dim = 1)
  
  # compute loss
  loss = criterion(y_pred , train_y_tensor)
  
  # initiate optimizer
  CBOW_linear_optimizer.zero_grad()
  CBOW_linear_optimizer_p.zero_grad()
  # backpropagation
  loss.backward()
  CBOW_linear_optimizer.step()
  CBOW_linear_optimizer_p.step()

  if i % 500 == 0:
    print(f'epoch {i} : {loss.item()}')

epoch 0 : 3.2166616916656494
epoch 500 : 2.713001251220703
epoch 1000 : 2.5400733947753906
epoch 1500 : 2.530947208404541
epoch 2000 : 2.5284221172332764
epoch 2500 : 2.5273563861846924
epoch 3000 : 2.526808738708496
epoch 3500 : 2.5264925956726074
epoch 4000 : 2.5262959003448486
epoch 4500 : 2.526167154312134
epoch 5000 : 2.5260801315307617


## Print lookuptable

In [81]:
for emb in CBOW_linear.parameters():
  print(emb)
for emb in CBOW_linear_p.parameters():
  print(emb)

Parameter containing:
tensor([[-1.3882, -1.5273, -1.3278, -1.4435, -1.3862, -1.5519, -1.5244, -1.5809,
         -1.3239, -1.4496, -1.5288, -1.6652, -1.4898, -1.5440, -0.9196, -1.4412,
         -1.5076, -1.5099, -1.6197, -1.6914, -1.7161, -1.4133, -1.6446, -1.6702,
         -1.6066],
        [ 1.6647,  1.7046,  1.7348,  1.5181,  1.7442,  1.4627,  1.5967,  1.7143,
          1.1617,  1.7504,  1.4620,  1.4617,  1.6521,  1.5347,  0.8969,  1.7253,
          1.4460,  1.4443,  1.5436,  1.6558,  1.4624,  1.5369,  1.4353,  1.5154,
          1.4628]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 1.2102, -1.9989],
        [-2.0631,  1.8450],
        [ 1.5625, -1.4628],
        [ 1.8891, -1.1678],
        [ 1.8306, -1.8809],
        [ 1.8420, -1.7414],
        [ 1.9208, -1.7551],
        [ 1.1312, -2.0279],
        [ 1.7208, -1.0484],
        [ 2.1885, -0.9130],
        [ 2.2013, -0.9102],
        [ 1.8098, -1.5049],
        [ 1.4105, -1.2777],
        [ 1.0903, -2.1824],
  